# UAVSAR Interferogram Processing

**Author**: Forrest Williams - Jet Propulsion Laboratory

This notebook provides a working example of end-to-end processing of UAVSAR interferograms through the ISCE application **stripmapApp.py**.

## Prep A: Initial setup of the notebook
Below we set up the directory structure for this notebook exercise. In addition, we load the required modules into our python environment using the **`import`** command.

In [ ]:
import glob
import os
import stat
import sys
from datetime import datetime
import itertools

## Defining the home and data directories at the processing location
tutorial_home_dir = os.path.abspath(os.getcwd())
work_dir = os.path.abspath(os.getcwd())
print("Tutorial directory: ", tutorial_home_dir)
print("Work directory: ", work_dir)

## Enter working directory
os.chdir(work_dir)

#set desired shell
os.environ["SHELL_NAME"] = 'sh'

#set environment variable for BBOX (SNWE)
#os.environ["BBOX"] = '36.976 37.57 -122.125 -121.2'
os.environ["BIG_BBOX"] = '36 39 -123 -120'

#set environment variable for download sh file
download_file = glob.glob(os.path.join(work_dir,'*.sh'))[0]
os.environ["DOWNLOAD_FILE"] = os.path.basename(download_file)

#set ISCE & stripmapApp environment variables
#os.environ["PATH"] += os.pathsep + path
os.environ["ISCE_HOME"] = '/home/jovyan/.local/envs/isce/lib/python3.9/site-packages/isce'
os.environ["PATH"] = os.path.join(os.environ["ISCE_HOME"],'applications') + os.pathsep + os.environ["PATH"]
os.environ["PATH"] = os.environ["PATH"] + os.pathsep + os.path.join(os.environ["ISCE_HOME"],'bin')
os.environ["PYTHONPATH"] = os.path.dirname(os.environ["ISCE_HOME"])
os.environ["PATH"] = os.environ["PATH"] + os.pathsep + '/home/jovyan/isce2/contrib/stack/stripmapStack'

## Prep B: Download data
First attempt to download data directly with the bash script.

In [ ]:
!chmod u+x $DOWNLOAD_FILE
!./$DOWNLOAD_FILE

In [ ]:
#set environment variable for .dop file
dop_file = glob.glob(os.path.join(work_dir,'*.dop'))[0]
os.environ["DOP_FILE"] = dop_file
#set environment variable for downloaded data prefix
os.environ["FILE_PREFIX"] = os.path.basename(dop_file).split('_')[0]

#set environment variable for segment number
segment_num = os.path.basename(glob.glob(os.path.join(work_dir,'*_1x1.slc'))[0])
segment_num = segment_num.split('_1x1.slc')[0].split('_')[-1].split('s')[-1]
os.environ["SEGMENT_NUM"] = segment_num

## Perform ISCE Processing

### Create Run Files

In [ ]:
!ls

In [3]:
!mkdir downloadHH
!mv $FILE_PREFIX* downloadHH/
!mv downloadHH/*.dop .

In [ ]:
!prepareUAVSAR_coregStack.py -i downloadHH -o SLC -d $DOP_FILE -s $SEGMENT_NUM

Some directory setup needs to be done before we can run `stackStripMap.py`:

In [ ]:
!ls

In [13]:
!mkdir merged
!ln -s $work_dir/SLC $work_dir/merged/SLC

Download DEM with `dem.py`:

In [21]:
os.mkdir(os.path.join(work_dir,'DEM'))
os.chdir(os.path.join(work_dir,'DEM'))

In [ ]:
## Download DEM (-f fill and -c ellipsoidal correction
!dem.py -a stitch -b $BIG_BBOX -k -s 1 -c -f

In [ ]:
## Set DEM path
os.chdir(work_dir)
dem_file = glob.glob(os.path.join('DEM','*.wgs84'))[0]
os.environ["DEM_FILE"] = os.path.abspath(os.path.join('DEM',dem_file))

Let's determine what the minimum time difference is:

In [ ]:
d = [str(os.path.basename(i)) for i in os.listdir('SLC') if os.path.isdir(os.path.join('SLC',i))]
dates = [datetime.strptime(x,'%Y%m%d') for x in d]
dates.sort()
difference = [x-y for x,y in zip(dates,[dates[0]]+dates[0:-1])]
#round to nearest 100
max_difference = max(difference).days
max_difference -= max_difference % -100
print(max_difference) #should be 800

#set environment variable
os.environ["TIME_DIFF"] = str(max_difference)

In [ ]:
#!stackStripMap.py -x "$BBOX" -s SLC/ -d $DEM_FILE -a 12 -r 3 -W interferogram --nofocus -S uavsar_stack -t $TIME_DIFF --filter_strength 0.2
!stackStripMap.py -s SLC/ -d $DEM_FILE -a 12 -r 3 -W interferogram --nofocus -S uavsar_stack -t $TIME_DIFF --filter_strength 0.2

In [ ]:
!ls run_files

In [ ]:
!nproc

### Excecute Run Files

For long-running scripts, it's better to run the commands within a shell with nohup specified instead of from inside the notebook because jupyter lab does not handle ssh disconnections well. Also, for **UAVSAR data only steps 1 and 8 need to be run** (coregister DEM and create interferograms). The cell below generates two .tsch that run each respectively and log the results. Copy the lines printed by this cell, then run them in the shell.

In [ ]:
#header = f'#!/bin/{os.environ["SHELL_NAME"]}\n\n'

# The crop step adjusts the order of the other steps
if os.path.exists(os.path.join(work_dir,'run_files','run_01_crop')):
    crop = f'cd run_files ; ./run_01_crop >& crop.log\n'
    master = f'cd run_files ; ./run_02_reference >& master.log\n'
    igrams = f'cd run_files ; ./run_09_igram >& igrams.log\n'
    cmds = {'crop':crop,'master':master,'igrams':igrams}
else:
    master = f'cd run_files ; ./run_01_reference >& master.log\n'
    igrams = f'cd run_files ; ./run_08_igram >& igrams.log\n'
    cmds = {'master':master,'igrams':igrams}

for key in cmds:
    name = f'run-{key}.{os.environ["SHELL_NAME"]}'
    with open(name, 'w') as f:
        #f.write(header)
        f.write(cmds[key])
    st = os.stat(name)
    os.chmod(name, st.st_mode | stat.S_IEXEC)
    print(f'./{name}')

nameOG = f'run-ALL.{os.environ["SHELL_NAME"]}'
with open(nameOG, 'w') as f:
    f.write('chmod u+x run_files/*\n\n')
    for key in cmds:
        name = f'run-{key}.{os.environ["SHELL_NAME"]}'
        f.write(f'./{name} >& {name}.log\n\n')
    st = os.stat(nameOG)
    os.chmod(nameOG, st.st_mode | stat.S_IEXEC)
    print(f'./{nameOG}')


### Run scripts

In [ ]:
!chmod u+x run-ALL.$SHELL_NAME
!./run-ALL.$SHELL_NAME >& run-ALL.log